- 데이터프레임은 subjectlist2_intextevents를 사용
- mv time 제거 안 한 버전
- 기존 테이블에서 int/ext의 결측치를 'unable로 생성한 int/ext'로 채워주고 생성할 수 있는 unable쌍도 추가한 테이블

In [33]:
import pandas as pd
from pandas import Series, DataFrame
import numpy as np
import pprint
from datetime import datetime, timedelta
import time
from dfply import *
import matplotlib.pyplot as plt
import seaborn as sns
import impyute as impy
import src.data_extraction.access_database as db
import src.data_extraction.filter_adult_patients as fap
import src.data_extraction.filter_ventilation_events as fve
import src.ventilator_mode_mapping.ventilator_mapping as vm
from src.utils import utils


In [20]:
subjectlist2_intextevents = pd.read_csv('./data/paired_df.csv', \
                                    parse_dates=['intubationtime', 'extubationtime', \
                                                 'admittime', 'dischtime', 'deathtime']) 
subjectlist2_intextevents

,Unnamed: 0,subject_id,hadm_id,stay_id,gender,los,int_stayid,admittime,intubationtime,int_itemid,...,reintubation_eventtime,reintubationtime,mvtime,final_event,ext_to_death,ext_to_disch,disch_to_death,class_code,class,stayid_mismatch
0,0,10001884,26184834,37510196,F,9.171817,37510196.0,2131-01-07 20:39:00,2131-01-11 04:30:00,224385.0,...,2131-01-13 04:00:00,620.0,False,False,10775.0,10775.0,0.0,211,failure,False
1,1,10001884,26184834,37510196,F,9.171817,37510196.0,2131-01-07 20:39:00,2131-01-13 04:00:00,224385.0,...,2131-01-14 06:59:00,478.0,True,False,9014.0,9014.0,0.0,211,failure,False
2,2,10001884,26184834,37510196,F,9.171817,37510196.0,2131-01-07 20:39:00,2131-01-14 06:59:00,224385.0,...,2131-01-19 00:00:00,1671.0,False,False,3426.0,3426.0,0.0,211,failure,False
3,3,10001884,26184834,37510196,F,9.171817,37510196.0,2131-01-07 20:39:00,2131-01-19 00:00:00,224385.0,...,NaN,NaN,True,True,1275.0,1275.0,0.0,22221,death,False
4,4,10003400,23559586,38383343,F,16.069838,38383343.0,2137-08-04 00:07:00,2137-08-17 21:21:00,224385.0,...,2137-08-25 02:00:00,4940.0,False,False,17365.0,17365.0,0.0,212,nonfailure,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13353,13436,19999068,21606769,30143796,M,6.765324,30143796.0,2161-08-24 04:10:00,2161-08-25 15:34:00,224385.0,...,NaN,NaN,False,True,NaN,7525.0,NaN,11,nonfailure,False
13354,13437,19999442,26785317,32336619,M,6.950370,32336619.0,2148-11-19 10:00:00,2148-11-19 19:00:00,224385.0,...,NaN,NaN,True,True,NaN,20290.0,NaN,11,nonfailure,False
13355,13438,19999840,21033226,38978960,M,5.297766,38978960.0,2164-09-10 13:47:00,2164-09-12 09:26:00,224385.0,...,2164-09-12 09:40:00,NaN,False,False,NaN,NaN,0.0,998,failure,False
13356,13439,19999840,21033226,38978960,M,5.297766,38978960.0,2164-09-10 13:47:00,2164-09-12 09:40:00,224385.0,...,2164-09-13 02:59:00,418.0,True,False,6821.0,6821.0,0.0,211,failure,False


####  필요한 컬럼만 가져오기 
- 사용할 컬럼 목록

- subject_id: 환자 고유번호 (출처: hosp_patients)
- hadm_id: 병원 입원 고유번호 (출처: hosp_admissions)
- stay_id: 중환자실 입원 고유번호. 발관 시간 기준으로 생성. 발관시간 NULL일 경우 삽관 이벤트 활용 (출처: icu_icustays)
- gender: 성별
- los: Length of Stay (days)
- admittime: 입원시각 (출처: hosp_admissions)
- intubationtime: 삽관시각 (출처: icu_procedureevents)
- extubationtime: 발관시각 (출처: icu_procedureevents) 
- extubationcause: 발관 사유 (Planned Extubation, Unplanned Extubation (patient-initiated), Unplanned Extubation (patient-uninitiated)) (출처: icu_procedureevents)
- dischtime: 퇴원시각 (출처: hosp_admissions)
- deathtime: 사망시각 (출처: hosp_admissions)
- seq_num: 삽관-발관 이벤트의 시퀀스 순서(1~n)
- total_seq_num: 각 hadm_id별 삽관-발관 시퀀스 중 마지막 번호(n) 
- mvtime: 삽관시각과 발관시각의 시간차 (단위: 분)
- reintubationtime: 발관 후 재삽관까지 걸린 시간 (단위: 분)
- mvtime_state (boolean): intext_duration이 1440분 이내 (<= 24시간)이면 mechanical ventilation (True)으로 분류 
- class_code: 케이스별로 고유한 13개 코드로 데이터를 분류 (하단 테이블 참조)
- class: class_code 기반으로 (Extubation) failure, non-failure, death의 3개 라벨로 분류함

In [22]:
# 필요한 컬럼만 가져오기 
# 사용할 컬럼 목록
columns_to_use = [
    'subject_id', 'hadm_id', 'stay_id', 'gender', 'los', 'admittime', 'intubationtime', 
    'extubationtime',  'extubationcause', 'dischtime', 'deathtime',  
    'seq_num', 'total_seq_num', 'mvtime', 'reintubationtime', 'mvtime_state', 
    'class_code', 'class'
]

# 필요한 컬럼들만 선택
subjectlist2_intextevents = subjectlist2_intextevents[columns_to_use]
subjectlist2_intextevents

,subject_id,hadm_id,stay_id,gender,los,admittime,intubationtime,extubationtime,extubationcause,dischtime,deathtime,seq_num,total_seq_num,intext_duration,reintubationtime,mvtime,class_code,class
0,10001884,26184834,37510196,F,9.171817,2131-01-07 20:39:00,2131-01-11 04:30:00,2131-01-12 17:40:00,Planned Extubation,2131-01-20 05:15:00,2131-01-20 05:15:00,1,4,2230.0,620.0,False,211,failure
1,10001884,26184834,37510196,F,9.171817,2131-01-07 20:39:00,2131-01-13 04:00:00,2131-01-13 23:01:00,Planned Extubation,2131-01-20 05:15:00,2131-01-20 05:15:00,2,4,1141.0,478.0,True,211,failure
2,10001884,26184834,37510196,F,9.171817,2131-01-07 20:39:00,2131-01-14 06:59:00,2131-01-17 20:09:00,Planned Extubation,2131-01-20 05:15:00,2131-01-20 05:15:00,3,4,5110.0,1671.0,False,211,failure
3,10001884,26184834,37510196,F,9.171817,2131-01-07 20:39:00,2131-01-19 00:00:00,2131-01-19 08:00:00,Planned Extubation,2131-01-20 05:15:00,2131-01-20 05:15:00,4,4,480.0,NaN,True,22221,death
4,10003400,23559586,38383343,F,16.069838,2137-08-04 00:07:00,2137-08-17 21:21:00,2137-08-21 15:40:00,Planned Extubation,2137-09-02 17:05:00,2137-09-02 17:05:00,1,2,5419.0,4940.0,False,212,nonfailure
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13353,19999068,21606769,30143796,M,6.765324,2161-08-24 04:10:00,2161-08-25 15:34:00,2161-08-28 13:35:00,Planned Extubation,2161-09-02 19:00:00,NaT,1,1,4201.0,NaN,False,11,nonfailure
13354,19999442,26785317,32336619,M,6.950370,2148-11-19 10:00:00,2148-11-19 19:00:00,2148-11-20 14:15:00,Planned Extubation,2148-12-04 16:25:00,NaT,1,1,1155.0,NaN,True,11,nonfailure
13355,19999840,21033226,38978960,M,5.297766,2164-09-10 13:47:00,2164-09-12 09:26:00,NaT,NaN,2164-09-17 13:42:00,2164-09-17 13:42:00,1,3,NaN,NaN,False,998,failure
13356,19999840,21033226,38978960,M,5.297766,2164-09-10 13:47:00,2164-09-12 09:40:00,2164-09-12 20:01:00,NaN,2164-09-17 13:42:00,2164-09-17 13:42:00,2,3,621.0,418.0,True,211,failure


In [ ]:
#### total_seq_num 생성
subjectlist2_intextevents['total_seq_num'] = subjectlist2_intextevents.groupby('hadm_id')['seq_num'].transform('max')
subjectlist2_intextevents.info()

-  변수 생성해주기 전에 int/ext nan 데이터 삭제 
1. intubationtime이 nan값인 행 갯수-> 907개
2. extubationtime이 nan값인 행 갯수 -> 939개 

In [23]:
subjectlist2_intextevents.intubationtime.isna().sum(), subjectlist2_intextevents.extubationtime.isna().sum()

(907, 939)

In [24]:
# int/ext중에 nan값이 있는 행은 삭제
subjectlist2_intextevents = subjectlist2_intextevents.dropna(subset=['intubationtime', 'extubationtime'])

- age , gender, los 컬럼 생성
- 이전 코드에서 테이블을 채울 수 있음에도 채워지지 않은 오류가 있음. 일단 여기서 채워두고 추후에 코드를 수정

In [25]:
import src.data_extraction.access_database as db
import src.data_extraction.filter_adult_patients as fap
import src.data_extraction.filter_ventilation_events as fve
from src.utils import utils
import psycopg2

# 데이터 저장위치
output_dir = './data'

# 데이터베이스 연결 설정
db_config = {
    'database': 'mimiciv',
    'user': 'mai_onlyselect',
    'password': 'student1q2w!@',
    'host': '1.212.63.162',
    'port': '35430'
}

# 호흡기 ITEM ID 설정 (웬만하면 건들지 마세요)
vent_ids_config = {
    'INTUBATION_ITEM_IDS': "224385",
    'EXTUBATION_ITEM_IDS': "225468, 225477, 227194"
}

# 테이블 쿼리 설정
tables_query = {
    'pg_tables': 'SELECT * FROM PG_TABLES;',
    'icustays': 'SELECT * FROM mimiciv_icu.icustays;',
    'admissions': 'SELECT * FROM mimiciv_hosp.admissions;'
}

db.print_config_info(db_config, tables_query)

# 데이터베이스 연결 설정
db_config = {
    'database': 'mimiciv',
    'user': 'mai_onlyselect',
    'password': 'student1q2w!@',
    'host': '1.212.63.162',
    'port': '35430'
}

try:
    # 데이터베이스 연결
    conn = psycopg2.connect(**db_config)
    print("Connected to the database successfully!")

    # 커서 생성
    curs = conn.cursor()

    if conn is not None:
        # 데이터베이스 작업 수행
        dataframes = db.retrieve_data(conn, tables_query)
    else:
        logging.error("Failed to connect to the database.")

    # SQL 쿼리 실행
    curs.execute("SELECT version();")

    # 결과 가져오기
    result = curs.fetchone()
    print("PostgreSQL version:", result)

except (psycopg2.Error, Exception) as error:
    print("Error while connecting to the database:", error)

INFO:root:Retrieved pg_tables: (164, 8)


--------- Database Configuration ---------
Database Name: mimiciv
User: mai_onlyselect
Password: *************
Host: 1.212.63.162
Port: 35430

--------- SQL Queries for Required Tables ---------
pg_tables: SELECT * FROM PG_TABLES;
icustays: SELECT * FROM mimiciv_icu.icustays;

Connected to the database successfully!


INFO:root:Retrieved icustays: (73181, 8)


PostgreSQL version: ('PostgreSQL 15.7 (Ubuntu 15.7-1.pgdg22.04+1) on x86_64-pc-linux-gnu, compiled by gcc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0, 64-bit',)


In [26]:
curs.execute("select * from mimiciv_hosp.patients;")
columns_name = [desc[0] for desc in curs.description]
patients = pd.DataFrame(curs.fetchall(), columns=columns_name)
patients

,subject_id,gender,anchor_age,anchor_year,anchor_year_group,dod
0,10000032,F,52,2180,2014 - 2016,2180-09-09
1,10000048,F,23,2126,2008 - 2010,None
2,10000068,F,19,2160,2008 - 2010,None
3,10000084,M,72,2160,2017 - 2019,2161-02-13
4,10000102,F,27,2136,2008 - 2010,None
...,...,...,...,...,...,...
299707,19999828,F,46,2147,2017 - 2019,None
299708,19999829,F,28,2186,2008 - 2010,None
299709,19999840,M,58,2164,2008 - 2010,2164-09-17
299710,19999914,F,49,2158,2017 - 2019,None


In [27]:
curs.execute("select * from mimiciv_icu.icustays;")
columns_name = [desc[0] for desc in curs.description]
icustays = pd.DataFrame(curs.fetchall(), columns=columns_name)
icustays

,subject_id,hadm_id,stay_id,first_careunit,last_careunit,intime,outtime,los
0,10000032,29079034,39553978,Medical Intensive Care Unit (MICU),Medical Intensive Care Unit (MICU),2180-07-23 14:00:00,2180-07-23 23:50:47,0.410266
1,10000980,26913865,39765666,Medical Intensive Care Unit (MICU),Medical Intensive Care Unit (MICU),2189-06-27 08:42:00,2189-06-27 20:38:27,0.497535
2,10001217,24597018,37067082,Surgical Intensive Care Unit (SICU),Surgical Intensive Care Unit (SICU),2157-11-20 19:18:02,2157-11-21 22:08:00,1.118032
3,10001217,27703517,34592300,Surgical Intensive Care Unit (SICU),Surgical Intensive Care Unit (SICU),2157-12-19 15:42:24,2157-12-20 14:27:41,0.948113
4,10001725,25563031,31205490,Medical/Surgical Intensive Care Unit (MICU/SICU),Medical/Surgical Intensive Care Unit (MICU/SICU),2110-04-11 15:52:22,2110-04-12 23:59:56,1.338588
...,...,...,...,...,...,...,...,...
73176,19999442,26785317,32336619,Surgical Intensive Care Unit (SICU),Surgical Intensive Care Unit (SICU),2148-11-19 14:23:43,2148-11-26 13:12:15,6.950370
73177,19999625,25304202,31070865,Medical/Surgical Intensive Care Unit (MICU/SICU),Medical/Surgical Intensive Care Unit (MICU/SICU),2139-10-10 19:18:00,2139-10-11 18:21:28,0.960741
73178,19999828,25744818,36075953,Medical Intensive Care Unit (MICU),Medical Intensive Care Unit (MICU),2149-01-08 18:12:00,2149-01-10 13:11:02,1.790995
73179,19999840,21033226,38978960,Trauma SICU (TSICU),Surgical Intensive Care Unit (SICU),2164-09-12 09:26:28,2164-09-17 16:35:15,5.297766


In [28]:
# merge 전에 기존의 'gender' 컬럼을 삭제하거나 무시하고 진행
subjectlist2_intextevents = subjectlist2_intextevents.drop(columns=['gender','los'], errors='ignore')

# merge 후 'patients' 데이터프레임의 'gender'와 'anchor_age'를 가져와서 병합
subjectlist2_intextevents = subjectlist2_intextevents.merge(patients[['subject_id', 'gender', 'anchor_age']], on='subject_id', how='left')

subjectlist2_intextevents = subjectlist2_intextevents.merge(icustays[['hadm_id', 'los']], on='hadm_id', how='left')

# Renaming 'anchor_age' to 'age' to match the requirement
subjectlist2_intextevents.rename(columns={'anchor_age': 'age'}, inplace=True)

subjectlist2_intextevents.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15756 entries, 0 to 15755
Data columns (total 19 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   subject_id        15756 non-null  int64         
 1   hadm_id           15756 non-null  int64         
 2   stay_id           15756 non-null  int64         
 3   admittime         15756 non-null  datetime64[ns]
 4   intubationtime    15756 non-null  datetime64[ns]
 5   extubationtime    15756 non-null  datetime64[ns]
 6   extubationcause   10250 non-null  object        
 7   dischtime         15756 non-null  datetime64[ns]
 8   deathtime         4485 non-null   datetime64[ns]
 9   seq_num           15756 non-null  int64         
 10  total_seq_num     15756 non-null  int64         
 11  intext_duration   15756 non-null  float64       
 12  reintubationtime  5939 non-null   float64       
 13  mvtime            15756 non-null  bool          
 14  class_code        1575

### 변수 추가 
- intubationtime과 extubationtime의 간격이 48시간 이상인 행의 변수 추가 
-> 변수의 charttime이 extubationtime으로부터 48시간 이전부터 extubationtime에 포함된 데이터
- intubationtime과 extubationtime의 간격이 48시간 미만인 행의 변수 추가 
-> 변수의 chartttime이 intubationtime부터 extubationtime에 포함된 데이터 
- 추가 순서 : abga, ventset, vital, gcs (순서는 중요하지 않음)


In [29]:
sub1_vitals = pd.read_csv('./data/sub1_vitals.csv')
sub1_gcs_notna = pd.read_csv('./data/sub1_gcs_notna.csv')
sub1_ventset = pd.read_csv('./data/sub1_ventset.csv')
sub1_abga_unique = pd.read_csv('./data/sub1_abga_unique.csv')

C:\Users\emati\AppData\Local\Temp\ipykernel_22248\923524036.py:4: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  sub1_abga_unique = pd.read_csv('./data/sub1_abga_unique.csv')


#### abga 추가 

In [42]:
subjectlist2_intextevents['intubationtime'] = pd.to_datetime(subjectlist2_intextevents['intubationtime'])
subjectlist2_intextevents['extubationtime'] = pd.to_datetime(subjectlist2_intextevents['extubationtime'])

sub1_abga_unique['charttime'] = pd.to_datetime(sub1_abga_unique['charttime'])
# itemid와 데이터 컬럼명을 매핑
itemid_to_column = {
    50802: 'Base excess',
    50817: 'SaO2',
    50818: 'pCO2',
    50820: 'pH',
    50821: 'pO2',
    50882: 'HCO3-'
}

# Step 2: 필터링 및 데이터 가져오기
results = []

for idx, row in subjectlist2_intextevents.iterrows():
    intubationtime = row['intubationtime']
    extubationtime = row['extubationtime']
    hadm_id = row['hadm_id']
    
    # 48시간 조건 적용
    time_diff = extubationtime - intubationtime
    if time_diff > timedelta(hours=48):
        intubationtime = extubationtime - timedelta(hours=48)
    
    # 해당 기간 동안의 데이터를 필터링
    mask = (sub1_abga_unique['hadm_id'] == hadm_id) & \
           (sub1_abga_unique['charttime'] >= intubationtime) & \
           (sub1_abga_unique['charttime'] <= extubationtime)
    
    filtered_data = sub1_abga_unique[mask]

    # Step 3: 각 항목에 맞는 데이터를 가져와서 결과를 정리
    for _, lab_row in filtered_data.iterrows():
        itemid = lab_row['itemid']
        column_name = itemid_to_column.get(itemid, None)
        if column_name:
            result_row = {
                'subject_id': row['subject_id'],
                'hadm_id': row['hadm_id'],
                'stay_id': row['stay_id'],
                'los': row['los'],
                'intubationtime': row['intubationtime'],
                'extubationtime': row['extubationtime'],
                'extubationcause': row['extubationcause'],
                'admittime': row['admittime'],
                'dischtime': row['dischtime'],
                'deathtime': row['deathtime'],
                'mvtime': row['mvtime'],
                'gender': row['gender'],
                'age': row['age'],  
                'reintubationtime' :row['reintubationtime'],
                'seq_num': row['seq_num'],
                'total_seq_num': row['total_seq_num'],
                'mvtime_state': row['mvtime_state'],
                'class_code': row['class_code'],
                'class': row['class'],
                'charttime': lab_row['charttime'],
                column_name: lab_row['valuenum'], 
            }
            results.append(result_row)

# Step 4: 결과 테이블 생성
abga_result_df = pd.DataFrame(results)

abga_result_df = abga_result_df.groupby(['hadm_id', 'charttime']).agg({
    'subject_id': 'first', #
    'stay_id': 'first',#
    'los': 'first',
    'gender': 'first',
    'dischtime':'first',
    'deathtime': 'first',
    'age': 'first',
    'intubationtime': 'first',
    'extubationtime': 'first',
    'extubationcause':'first',
    'seq_num': 'first',
    'total_seq_num': 'first',
    'mvtime_state': 'first',
    'class_code': 'first',
    'class': 'first',
    'reintubationtime': 'first',
    'pCO2': lambda x: x.dropna().iloc[0] if not x.dropna().empty else None,
    'pO2': lambda x: x.dropna().iloc[0] if not x.dropna().empty else None,
    'SaO2': lambda x: x.dropna().iloc[0] if not x.dropna().empty else None,
    'HCO3-': lambda x: x.dropna().iloc[0] if not x.dropna().empty else None,
    'pH': lambda x: x.dropna().iloc[0] if not x.dropna().empty else None,
    'Base excess': lambda x: x.dropna().iloc[0] if not x.dropna().empty else None
}).reset_index()


# 필요한 컬럼들만 출력
final_columns = ['subject_id', 'hadm_id','los', 'stay_id',  'gender', 'age', 'admittime','dischtime', 'deathtime','intubationtime', 'extubationtime' ,'extubationcause','seq_num','total_seq_num','mvtime','class_code','class'
,'reintubationtime','charttime','pCO2', 'pO2', 'SaO2', 'HCO3-', 'pH', 'Base excess']
abga_result_df = abga_result_df.reindex(columns=final_columns, fill_value=float('nan'))
abga_result_df

,subject_id,hadm_id,los,stay_id,gender,age,admittime,dischtime,deathtime,intubationtime,extubationtime,extubationcause,seq_num,total_seq_num,mvtime,class_code,class,reintubationtime,charttime,pCO2,pO2,SaO2,HCO3-,pH,Base excess
0,14577567,20001361,6.046644,33475095,M,30,NaN,2143-05-18 16:58:00,NaT,2143-05-04 16:59:00,2143-05-05 08:30:00,Planned Extubation,1,3,True,211,failure,690.0,2143-05-04 17:24:00,NaN,NaN,NaN,22.0,NaN,NaN
1,14577567,20001361,6.046644,33475095,M,30,NaN,2143-05-18 16:58:00,NaT,2143-05-04 16:59:00,2143-05-05 08:30:00,Planned Extubation,1,3,True,211,failure,690.0,2143-05-04 17:38:00,NaN,NaN,NaN,NaN,7.26,NaN
2,14577567,20001361,6.046644,33475095,M,30,NaN,2143-05-18 16:58:00,NaT,2143-05-04 16:59:00,2143-05-05 08:30:00,Planned Extubation,1,3,True,211,failure,690.0,2143-05-04 18:01:00,40.0,59.0,NaN,NaN,7.33,-4.0
3,14577567,20001361,6.046644,33475095,M,30,NaN,2143-05-18 16:58:00,NaT,2143-05-04 16:59:00,2143-05-05 08:30:00,Planned Extubation,1,3,True,211,failure,690.0,2143-05-04 21:07:00,NaN,NaN,NaN,20.0,NaN,NaN
4,14577567,20001361,6.046644,33475095,M,30,NaN,2143-05-18 16:58:00,NaT,2143-05-04 16:59:00,2143-05-05 08:30:00,Planned Extubation,1,3,True,211,failure,690.0,2143-05-04 22:19:00,40.0,94.0,NaN,NaN,7.32,-5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69372,17660536,29997844,3.110880,34886849,M,81,NaN,2125-03-03 16:21:00,NaT,2125-02-26 16:25:00,2125-02-27 00:01:00,None,2,2,True,221,nonfailure,NaN,2125-02-26 20:34:00,35.0,171.0,NaN,NaN,7.33,-6.0
69373,17660536,29997844,3.110880,34886849,M,81,NaN,2125-03-03 16:21:00,NaT,2125-02-26 16:25:00,2125-02-27 00:01:00,None,2,2,True,221,nonfailure,NaN,2125-02-26 23:23:00,33.0,173.0,NaN,NaN,7.35,-6.0
69374,15129243,29999098,11.071678,33475199,M,76,NaN,2128-06-20 11:25:00,2128-06-20 11:25:00,2128-06-10 09:30:00,2128-06-11 08:25:00,None,2,2,True,221,nonfailure,NaN,2128-06-10 10:52:00,34.0,101.0,NaN,NaN,7.43,0.0
69375,15129243,29999098,11.071678,33475199,M,76,NaN,2128-06-20 11:25:00,2128-06-20 11:25:00,2128-06-10 09:30:00,2128-06-11 08:25:00,None,2,2,True,221,nonfailure,NaN,2128-06-10 19:02:00,33.0,96.0,NaN,NaN,7.43,0.0


#### ventset추가 

In [34]:
ventset_patient = vm.update_ventilator_scoring(sub1_ventset)
ventset_patient

,subject_id,hadm_id,stay_id,charttime,respiratory_rate_set,respiratory_rate_total,respiratory_rate_spontaneous,minute_volume,tidal_volume_set,tidal_volume_observed,tidal_volume_spontaneous,plateau_pressure,peep,fio2,flow_rate,ventilator_mode,ventilator_mode_hamilton,ventilator_type,ventilator_scoring
0,10001884,26184834,37510196,2131-01-11 04:00:00,20.0,20.0,0.0,9.0,450.0,479.0,NaN,24.0,5.0,100.0,39.9,NaN,APV (cmv),Hamilton,4.0
1,10001884,26184834,37510196,2131-01-11 07:00:00,20.0,20.0,0.0,8.0,400.0,408.0,NaN,22.0,5.1,50.0,35.6,NaN,APV (cmv),Hamilton,4.0
2,10001884,26184834,37510196,2131-01-11 11:00:00,20.0,20.0,0.0,8.3,400.0,430.0,NaN,23.0,5.2,50.0,32.3,NaN,APV (cmv),Hamilton,4.0
3,10001884,26184834,37510196,2131-01-11 12:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,50.0,NaN,NaN,NaN,NaN,NaN
4,10001884,26184834,37510196,2131-01-11 14:00:00,NaN,NaN,NaN,7.0,NaN,331.0,331.0,NaN,5.0,NaN,NaN,NaN,SPONT,NaN,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
568379,19999840,21033226,38978960,2164-09-16 23:00:00,14.0,17.0,0.0,11.4,500.0,508.0,NaN,27.0,10.0,60.0,NaN,CMV/ASSIST/AutoFlow,NaN,Drager,4.0
568380,19999840,21033226,38978960,2164-09-17 01:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.0,40.0,NaN,NaN,NaN,Drager,NaN
568381,19999840,21033226,38978960,2164-09-17 04:00:00,14.0,19.0,0.0,9.1,500.0,503.0,NaN,28.0,13.0,40.0,NaN,CMV/ASSIST/AutoFlow,NaN,Drager,4.0
568382,19999840,21033226,38978960,2164-09-17 08:00:00,14.0,23.0,9.0,11.5,500.0,500.0,NaN,27.0,12.0,40.0,NaN,CMV/ASSIST,NaN,Drager,4.0


In [36]:
pd.set_option('display.max_columns', None) # 모든 열을 볼 수 있도록 설정

In [37]:
ventset_patient['charttime'] = pd.to_datetime(ventset_patient['charttime'])
# Step 1: Group ventset_patient by 'hadm_id' once before the loop
ventset_patient_grouped = ventset_patient.groupby('hadm_id')

# Step 2: Initialize an empty result list
optimized_result_list = []

# Step 3: Process each row in subjectlist2_intextevents
for index, row in subjectlist2_intextevents.iterrows():
    hadm_id = row['hadm_id']
    intubationtime = row['intubationtime']
    extubationtime = row['extubationtime']
    
    # Determine start_time based on time difference
    time_diff = extubationtime - intubationtime
    start_time = extubationtime - timedelta(hours=48) if time_diff > timedelta(hours=48) else intubationtime

    # Check if hadm_id exists in ventset_patient_grouped
    if hadm_id in ventset_patient_grouped.groups:
        # Retrieve relevant group for the hadm_id (faster than filtering the entire dataset)
        group = ventset_patient_grouped.get_group(hadm_id)
        
        # Step 4: Filter the group by time window only (much smaller subset than the whole dataset)
        filtered_group = group[
            (group['charttime'] >= start_time) &
            (group['charttime'] <= extubationtime)
        ]
        
        # Step 5: Append filtered rows to result
        for _, vent_row in filtered_group.iterrows():
            new_row = row.copy()
            for col in [
                'charttime', 'respiratory_rate_set', 'respiratory_rate_total',
                'respiratory_rate_spontaneous', 'minute_volume', 'tidal_volume_set',
                'tidal_volume_observed', 'tidal_volume_spontaneous', 'plateau_pressure',
                'peep', 'fio2', 'flow_rate', 'ventilator_mode', 
                'ventilator_mode_hamilton', 'ventilator_type', 'ventilator_scoring'
            ]:
                new_row[col] = vent_row.get(col, None)  # Using get to avoid KeyErrors
            optimized_result_list.append(new_row)

# Step 6: Create the final result DataFrame
ventset_result_df = pd.DataFrame(optimized_result_list)
ventset_result_df

,subject_id,hadm_id,stay_id,admittime,intubationtime,extubationtime,extubationcause,dischtime,deathtime,seq_num,total_seq_num,intext_duration,reintubationtime,mvtime,class_code,class,gender,age,los,charttime,respiratory_rate_set,respiratory_rate_total,respiratory_rate_spontaneous,minute_volume,tidal_volume_set,tidal_volume_observed,tidal_volume_spontaneous,plateau_pressure,peep,fio2,flow_rate,ventilator_mode,ventilator_mode_hamilton,ventilator_type,ventilator_scoring
0,10001884,26184834,37510196,2131-01-07 20:39:00,2131-01-11 04:30:00,2131-01-12 17:40:00,Planned Extubation,2131-01-20 05:15:00,2131-01-20 05:15:00,1,4,2230.0,620.0,False,211,failure,F,68,9.171817,2131-01-11 07:00:00,20.0,20.0,0.0,8.0,400.0,408.0,NaN,22.0,5.1,50.0,35.6,NaN,APV (cmv),Hamilton,4.0
0,10001884,26184834,37510196,2131-01-07 20:39:00,2131-01-11 04:30:00,2131-01-12 17:40:00,Planned Extubation,2131-01-20 05:15:00,2131-01-20 05:15:00,1,4,2230.0,620.0,False,211,failure,F,68,9.171817,2131-01-11 11:00:00,20.0,20.0,0.0,8.3,400.0,430.0,NaN,23.0,5.2,50.0,32.3,NaN,APV (cmv),Hamilton,4.0
0,10001884,26184834,37510196,2131-01-07 20:39:00,2131-01-11 04:30:00,2131-01-12 17:40:00,Planned Extubation,2131-01-20 05:15:00,2131-01-20 05:15:00,1,4,2230.0,620.0,False,211,failure,F,68,9.171817,2131-01-11 12:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,50.0,NaN,NaN,NaN,NaN,NaN
0,10001884,26184834,37510196,2131-01-07 20:39:00,2131-01-11 04:30:00,2131-01-12 17:40:00,Planned Extubation,2131-01-20 05:15:00,2131-01-20 05:15:00,1,4,2230.0,620.0,False,211,failure,F,68,9.171817,2131-01-11 14:00:00,NaN,NaN,NaN,7.0,NaN,331.0,331.0,NaN,5.0,NaN,NaN,NaN,SPONT,NaN,1.0
0,10001884,26184834,37510196,2131-01-07 20:39:00,2131-01-11 04:30:00,2131-01-12 17:40:00,Planned Extubation,2131-01-20 05:15:00,2131-01-20 05:15:00,1,4,2230.0,620.0,False,211,failure,F,68,9.171817,2131-01-11 16:00:00,NaN,19.0,19.0,6.6,NaN,351.0,351.0,NaN,5.0,50.0,36.0,NaN,SPONT,Hamilton,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15755,19999840,21033226,38978960,2164-09-10 13:47:00,2164-09-13 02:59:00,2164-09-17 12:07:00,NaN,2164-09-17 13:42:00,2164-09-17 13:42:00,3,3,6308.0,NaN,False,22221,death,M,58,5.297766,2164-09-16 23:00:00,14.0,17.0,0.0,11.4,500.0,508.0,NaN,27.0,10.0,60.0,NaN,CMV/ASSIST/AutoFlow,NaN,Drager,4.0
15755,19999840,21033226,38978960,2164-09-10 13:47:00,2164-09-13 02:59:00,2164-09-17 12:07:00,NaN,2164-09-17 13:42:00,2164-09-17 13:42:00,3,3,6308.0,NaN,False,22221,death,M,58,5.297766,2164-09-17 01:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.0,40.0,NaN,NaN,NaN,Drager,NaN
15755,19999840,21033226,38978960,2164-09-10 13:47:00,2164-09-13 02:59:00,2164-09-17 12:07:00,NaN,2164-09-17 13:42:00,2164-09-17 13:42:00,3,3,6308.0,NaN,False,22221,death,M,58,5.297766,2164-09-17 04:00:00,14.0,19.0,0.0,9.1,500.0,503.0,NaN,28.0,13.0,40.0,NaN,CMV/ASSIST/AutoFlow,NaN,Drager,4.0
15755,19999840,21033226,38978960,2164-09-10 13:47:00,2164-09-13 02:59:00,2164-09-17 12:07:00,NaN,2164-09-17 13:42:00,2164-09-17 13:42:00,3,3,6308.0,NaN,False,22221,death,M,58,5.297766,2164-09-17 08:00:00,14.0,23.0,9.0,11.5,500.0,500.0,NaN,27.0,12.0,40.0,NaN,CMV/ASSIST,NaN,Drager,4.0


#### 병합 (abga를 변수로 추가한 테이블과 ventset를 변수로 추가한 테이블과 병합)

In [43]:
# Concatenate the two dataframes
combined_df = pd.concat([abga_result_df, ventset_result_df], ignore_index=True)

# Group by 'hadm_id' and 'charttime' and combine the columns
abga_ventset = combined_df.groupby(['hadm_id', 'charttime'], as_index=False).first()

# Add missing columns from both dataframes (in case one is missing a column from the other)
columns_to_add = ['pCO2', 'pO2', 'SaO2', 'HCO3-', 'pH', 'Base excess','respiratory_rate_set', 'respiratory_rate_total',
                'respiratory_rate_spontaneous', 'minute_volume', 'tidal_volume_set',
                'tidal_volume_observed', 'tidal_volume_spontaneous', 'plateau_pressure',
                'peep', 'fio2', 'flow_rate', 'ventilator_mode', 
                'ventilator_mode_hamilton', 'ventilator_type', 'ventilator_scoring']
for col in columns_to_add:
    if col not in abga_ventset.columns:
        abga_ventset[col] = float('nan')
abga_ventset

C:\Users\emati\AppData\Local\Temp\ipykernel_22248\1802575858.py:2: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  combined_df = pd.concat([abga_result_df, ventset_result_df], ignore_index=True)


,hadm_id,charttime,subject_id,los,stay_id,gender,age,admittime,dischtime,deathtime,intubationtime,extubationtime,extubationcause,seq_num,total_seq_num,mvtime,class_code,class,reintubationtime,pCO2,pO2,SaO2,HCO3-,pH,Base excess,intext_duration,respiratory_rate_set,respiratory_rate_total,respiratory_rate_spontaneous,minute_volume,tidal_volume_set,tidal_volume_observed,tidal_volume_spontaneous,plateau_pressure,peep,fio2,flow_rate,ventilator_mode,ventilator_mode_hamilton,ventilator_type,ventilator_scoring
0,20001361,2143-05-04 17:00:00,14577567,6.046644,33475095,M,30,2143-05-04 14:55:00,2143-05-18 16:58:00,NaT,2143-05-04 16:59:00,2143-05-05 08:30:00,Planned Extubation,1,3,True,211,failure,690.0,NaN,NaN,NaN,NaN,NaN,NaN,931.0,16.0,22.0,6.0,NaN,650.0,810.0,NaN,17.0,5.0,100.0,NaN,CMV/AutoFlow,None,Drager,4.0
1,20001361,2143-05-04 17:04:00,14577567,6.046644,33475095,M,30,2143-05-04 14:55:00,2143-05-18 16:58:00,NaT,2143-05-04 16:59:00,2143-05-05 08:30:00,Planned Extubation,1,3,True,211,failure,690.0,NaN,NaN,NaN,NaN,NaN,NaN,931.0,NaN,NaN,NaN,13.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,NaN
2,20001361,2143-05-04 17:24:00,14577567,6.046644,33475095,M,30,NaT,2143-05-18 16:58:00,NaT,2143-05-04 16:59:00,2143-05-05 08:30:00,Planned Extubation,1,3,True,211,failure,690.0,NaN,NaN,NaN,22.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,NaN
3,20001361,2143-05-04 17:38:00,14577567,6.046644,33475095,M,30,NaT,2143-05-18 16:58:00,NaT,2143-05-04 16:59:00,2143-05-05 08:30:00,Planned Extubation,1,3,True,211,failure,690.0,NaN,NaN,NaN,NaN,7.26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,NaN
4,20001361,2143-05-04 18:00:00,14577567,6.046644,33475095,M,30,2143-05-04 14:55:00,2143-05-18 16:58:00,NaT,2143-05-04 16:59:00,2143-05-05 08:30:00,Planned Extubation,1,3,True,211,failure,690.0,NaN,NaN,NaN,NaN,NaN,NaN,931.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.0,NaN,NaN,None,None,None,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196743,29999098,2128-06-10 19:02:00,15129243,11.071678,33475199,M,76,NaT,2128-06-20 11:25:00,2128-06-20 11:25:00,2128-06-10 09:30:00,2128-06-11 08:25:00,None,2,2,True,221,nonfailure,NaN,33.0,96.0,NaN,NaN,7.43,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,NaN
196744,29999098,2128-06-11 00:00:00,15129243,11.071678,33475199,M,76,2128-06-09 23:45:00,2128-06-20 11:25:00,2128-06-20 11:25:00,2128-06-10 09:30:00,2128-06-11 08:25:00,None,2,2,True,221,nonfailure,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1375.0,16.0,32.0,0.0,14.8,400.0,448.0,NaN,0.0,8.0,40.0,NaN,CMV/ASSIST/AutoFlow,None,Drager,4.0
196745,29999098,2128-06-11 03:38:00,15129243,11.071678,33475199,M,76,NaT,2128-06-20 11:25:00,2128-06-20 11:25:00,2128-06-10 09:30:00,2128-06-11 08:25:00,None,2,2,True,221,nonfailure,NaN,NaN,NaN,NaN,21.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,NaN
196746,29999098,2128-06-11 04:00:00,15129243,11.071678,33475199,M,76,2128-06-09 23:45:00,2128-06-20 11:25:00,2128-06-20 11:25:00,2128-06-10 09:30:00,2128-06-11 08:25:00,None,2,2,True,221,nonfailure,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1375.0,16.0,25.0,0.0,12.2,400.0,568.0,NaN,14.0,8.0,40.0,NaN,CMV/ASSIST/AutoFlow,None,Drager,4.0


#### vital_sign 추가

In [28]:
sub1_vitals

,subject_id,hadm_id,stay_id,charttime,heart_rate,sbp,dbp,mbp,sbp_ni,dbp_ni,mbp_ni,resp_rate,temperature,temperature_site,spo2,glucose
0,10001884,26184834,37510196,2131-01-11 04:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.0,NaN,NaN,NaN,NaN
1,10001884,26184834,37510196,2131-01-11 04:20:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.0,NaN,NaN,100.0,NaN
2,10001884,26184834,37510196,2131-01-11 04:22:00,38.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10001884,26184834,37510196,2131-01-11 04:25:00,NaN,180.0,12.0,46.0,180.0,12.0,46.0,NaN,NaN,NaN,NaN,NaN
4,10001884,26184834,37510196,2131-01-11 05:00:00,60.0,NaN,NaN,NaN,NaN,NaN,NaN,10.0,NaN,NaN,98.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3384063,19999840,21033226,38978960,2164-09-17 13:17:00,NaN,NaN,NaN,44.0,NaN,NaN,44.0,NaN,NaN,NaN,NaN,NaN
3384064,19999840,21033226,38978960,2164-09-17 13:18:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,139.0
3384065,19999840,21033226,38978960,2164-09-17 13:20:00,74.0,NaN,NaN,NaN,NaN,NaN,NaN,29.0,NaN,NaN,NaN,NaN
3384066,19999840,21033226,38978960,2164-09-17 13:34:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,275.0


In [38]:
import pandas as pd
from datetime import timedelta

sub1_vitals['charttime'] = pd.to_datetime(sub1_vitals['charttime'])
# Step 1: Split 'sub1_vitals' by 'hadm_id' into chunks
hadm_ids = sub1_vitals['hadm_id'].unique()

# 원하는 청크 크기 설정 (예: 한 번에 100개 'hadm_id'씩 처리)
chunk_size = 1000

# Step 2: Initialize an empty result list
optimized_result_list = []

# Step 3: Process each chunk
for i in range(0, len(hadm_ids), chunk_size):
    # 청크별로 'hadm_id' 나누기
    chunk_hadm_ids = hadm_ids[i:i + chunk_size]
    
    # Step 4: 해당 청크에 해당하는 'sub1_vitals' 데이터만 가져오기
    sub1_vitals_chunk = sub1_vitals[sub1_vitals['hadm_id'].isin(chunk_hadm_ids)]
    
    # Step 5: Group the chunk by 'hadm_id'
    sub1_vitals_grouped = sub1_vitals_chunk.groupby('hadm_id')

    # Step 6: Process each row in 'subjectlist2_intextevents'
    for index, row in subjectlist2_intextevents.iterrows():
        hadm_id = row['hadm_id']
        intubationtime = row['intubationtime']
        extubationtime = row['extubationtime']
        
        # Determine start_time based on time difference
        time_diff = extubationtime - intubationtime
        start_time = extubationtime - timedelta(hours=48) if time_diff > timedelta(hours=48) else intubationtime

        # Check if hadm_id exists in the current chunk
        if hadm_id in sub1_vitals_grouped.groups:
            # Retrieve relevant group for the hadm_id (faster than filtering the entire dataset)
            group = sub1_vitals_grouped.get_group(hadm_id)
            
            # Step 7: Filter the group by time window only (much smaller subset than the whole dataset)
            filtered_group = group[
                (group['charttime'] >= start_time) &
                (group['charttime'] <= extubationtime)
            ]
            
            # Step 8: Append filtered rows to result
            for _, vital_row in filtered_group.iterrows():
                new_row = row.copy()
                for col in [
                    'charttime', 'heart_rate', 'sbp', 'dbp', 'mbp', 
                    'resp_rate', 'temperature', 'temperature_site', 'spo2', 'glucose'
                ]:
                    new_row[col] = vital_row.get(col, None)  # Using get to avoid KeyErrors
                optimized_result_list.append(new_row)

# Step 9: Create the final result DataFrame
vitals_result_df = pd.DataFrame(optimized_result_list)

# Display the final result
vitals_result_df

,subject_id,hadm_id,stay_id,admittime,intubationtime,extubationtime,extubationcause,dischtime,deathtime,seq_num,total_seq_num,intext_duration,reintubationtime,mvtime,class_code,class,gender,age,los,charttime,heart_rate,sbp,dbp,mbp,resp_rate,temperature,temperature_site,spo2,glucose
0,10001884,26184834,37510196,2131-01-07 20:39:00,2131-01-11 04:30:00,2131-01-12 17:40:00,Planned Extubation,2131-01-20 05:15:00,2131-01-20 05:15:00,1,4,2230.0,620.0,False,211,failure,F,68,9.171817,2131-01-11 05:00:00,60.0,NaN,NaN,NaN,10.0,NaN,NaN,98.0,NaN
0,10001884,26184834,37510196,2131-01-07 20:39:00,2131-01-11 04:30:00,2131-01-12 17:40:00,Planned Extubation,2131-01-20 05:15:00,2131-01-20 05:15:00,1,4,2230.0,620.0,False,211,failure,F,68,9.171817,2131-01-11 05:01:00,NaN,167.0,49.0,70.0,NaN,NaN,NaN,NaN,NaN
0,10001884,26184834,37510196,2131-01-07 20:39:00,2131-01-11 04:30:00,2131-01-12 17:40:00,Planned Extubation,2131-01-20 05:15:00,2131-01-20 05:15:00,1,4,2230.0,620.0,False,211,failure,F,68,9.171817,2131-01-11 06:00:00,72.0,NaN,NaN,NaN,20.0,NaN,NaN,100.0,NaN
0,10001884,26184834,37510196,2131-01-07 20:39:00,2131-01-11 04:30:00,2131-01-12 17:40:00,Planned Extubation,2131-01-20 05:15:00,2131-01-20 05:15:00,1,4,2230.0,620.0,False,211,failure,F,68,9.171817,2131-01-11 06:01:00,NaN,102.0,63.0,75.0,NaN,NaN,NaN,NaN,NaN
0,10001884,26184834,37510196,2131-01-07 20:39:00,2131-01-11 04:30:00,2131-01-12 17:40:00,Planned Extubation,2131-01-20 05:15:00,2131-01-20 05:15:00,1,4,2230.0,620.0,False,211,failure,F,68,9.171817,2131-01-11 06:31:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,148.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15755,19999840,21033226,38978960,2164-09-10 13:47:00,2164-09-13 02:59:00,2164-09-17 12:07:00,NaN,2164-09-17 13:42:00,2164-09-17 13:42:00,3,3,6308.0,NaN,False,22221,death,M,58,5.297766,2164-09-17 10:00:00,61.0,111.0,52.0,66.0,23.0,NaN,NaN,96.0,NaN
15755,19999840,21033226,38978960,2164-09-10 13:47:00,2164-09-13 02:59:00,2164-09-17 12:07:00,NaN,2164-09-17 13:42:00,2164-09-17 13:42:00,3,3,6308.0,NaN,False,22221,death,M,58,5.297766,2164-09-17 11:00:00,62.0,79.0,42.0,50.0,NaN,NaN,NaN,93.0,NaN
15755,19999840,21033226,38978960,2164-09-10 13:47:00,2164-09-13 02:59:00,2164-09-17 12:07:00,NaN,2164-09-17 13:42:00,2164-09-17 13:42:00,3,3,6308.0,NaN,False,22221,death,M,58,5.297766,2164-09-17 11:01:00,NaN,NaN,NaN,NaN,20.0,NaN,NaN,NaN,NaN
15755,19999840,21033226,38978960,2164-09-10 13:47:00,2164-09-13 02:59:00,2164-09-17 12:07:00,NaN,2164-09-17 13:42:00,2164-09-17 13:42:00,3,3,6308.0,NaN,False,22221,death,M,58,5.297766,2164-09-17 11:30:00,63.0,108.0,57.0,67.0,24.0,NaN,NaN,92.0,NaN


In [41]:
# vitals_result_df.to_csv('./data/vitals_result_df.csv') 

#### 병합

In [44]:
# Concatenate the two dataframes
combined_df = pd.concat([abga_ventset, vitals_result_df], ignore_index=True)

# Group by 'hadm_id' and 'charttime' and combine the columns
abga_ventset_vital = combined_df.groupby(['hadm_id', 'charttime'], as_index=False).first()

# Add missing columns from both dataframes (in case one is missing a column from the other)
columns_to_add = ['pCO2', 'pO2', 'SaO2', 'HCO3-', 'pH', 'Base excess',
                  
                'respiratory_rate_set', 'respiratory_rate_total', 'respiratory_rate_spontaneous', 'minute_volume', 'tidal_volume_set',
                'tidal_volume_observed', 'tidal_volume_spontaneous', 'plateau_pressure','peep', 'fio2', 'flow_rate', 
                  'ventilator_mode', 'ventilator_mode_hamilton', 'ventilator_type', 'ventilator_scoring',
                  
                'heart_rate','sbp'	,'dbp','mbp','resp_rate',	'temperature',	'temperature_site',	'spo2',	'glucose']
for col in columns_to_add:
    if col not in abga_ventset_vital.columns:
        abga_ventset_vital[col] = float('nan')
abga_ventset_vital

,hadm_id,charttime,subject_id,los,stay_id,gender,age,admittime,dischtime,deathtime,intubationtime,extubationtime,extubationcause,seq_num,total_seq_num,mvtime,class_code,class,reintubationtime,pCO2,pO2,SaO2,HCO3-,pH,Base excess,intext_duration,respiratory_rate_set,respiratory_rate_total,respiratory_rate_spontaneous,minute_volume,tidal_volume_set,tidal_volume_observed,tidal_volume_spontaneous,plateau_pressure,peep,fio2,flow_rate,ventilator_mode,ventilator_mode_hamilton,ventilator_type,ventilator_scoring,heart_rate,sbp,dbp,mbp,resp_rate,temperature,temperature_site,spo2,glucose
0,20001361,2143-05-04 17:00:00,14577567,6.046644,33475095,M,30,2143-05-04 14:55:00,2143-05-18 16:58:00,NaT,2143-05-04 16:59:00,2143-05-05 08:30:00,Planned Extubation,1,3,True,211,failure,690.0,NaN,NaN,NaN,NaN,NaN,NaN,931.0,16.0,22.0,6.0,NaN,650.0,810.0,NaN,17.0,5.0,100.0,NaN,CMV/AutoFlow,None,Drager,4.0,NaN,NaN,NaN,NaN,22.0,NaN,None,NaN,NaN
1,20001361,2143-05-04 17:04:00,14577567,6.046644,33475095,M,30,2143-05-04 14:55:00,2143-05-18 16:58:00,NaT,2143-05-04 16:59:00,2143-05-05 08:30:00,Planned Extubation,1,3,True,211,failure,690.0,NaN,NaN,NaN,NaN,NaN,NaN,931.0,NaN,NaN,NaN,13.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN
2,20001361,2143-05-04 17:09:00,14577567,6.046644,33475095,M,30,2143-05-04 14:55:00,2143-05-18 16:58:00,NaT,2143-05-04 16:59:00,2143-05-05 08:30:00,Planned Extubation,1,3,True,211,failure,690.0,NaN,NaN,NaN,NaN,NaN,NaN,931.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,NaN,NaN,124.0,86.0,95.0,NaN,NaN,None,NaN,NaN
3,20001361,2143-05-04 17:10:00,14577567,6.046644,33475095,M,30,2143-05-04 14:55:00,2143-05-18 16:58:00,NaT,2143-05-04 16:59:00,2143-05-05 08:30:00,Planned Extubation,1,3,True,211,failure,690.0,NaN,NaN,NaN,NaN,NaN,NaN,931.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,NaN,NaN,NaN,NaN,NaN,16.0,NaN,None,NaN,NaN
4,20001361,2143-05-04 17:11:00,14577567,6.046644,33475095,M,30,2143-05-04 14:55:00,2143-05-18 16:58:00,NaT,2143-05-04 16:59:00,2143-05-05 08:30:00,Planned Extubation,1,3,True,211,failure,690.0,NaN,NaN,NaN,NaN,NaN,NaN,931.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,NaN,78.0,NaN,NaN,NaN,16.0,NaN,None,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
605617,29999098,2128-06-11 04:00:00,15129243,11.071678,33475199,M,76,2128-06-09 23:45:00,2128-06-20 11:25:00,2128-06-20 11:25:00,2128-06-10 09:30:00,2128-06-11 08:25:00,None,2,2,True,221,nonfailure,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1375.0,16.0,25.0,0.0,12.2,400.0,568.0,NaN,14.0,8.0,40.0,NaN,CMV/ASSIST/AutoFlow,None,Drager,4.0,68.0,130.0,52.0,75.0,22.5,37.50,Oral,98.0,NaN
605618,29999098,2128-06-11 05:00:00,15129243,11.071678,33475199,M,76,2128-06-09 23:45:00,2128-06-20 11:25:00,2128-06-20 11:25:00,2128-06-10 09:30:00,2128-06-11 08:25:00,None,2,2,True,221,nonfailure,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1375.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,NaN,89.0,155.0,63.0,93.0,19.0,NaN,None,100.0,NaN
605619,29999098,2128-06-11 06:00:00,15129243,11.071678,33475199,M,76,2128-06-09 23:45:00,2128-06-20 11:25:00,2128-06-20 11:25:00,2128-06-10 09:30:00,2128-06-11 08:25:00,None,2,2,True,221,nonfailure,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1375.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,NaN,75.0,130.0,50.0,74.0,16.0,NaN,None,97.0,NaN
605620,29999098,2128-06-11 07:00:00,15129243,11.071678,33475199,M,76,2128-06-09 23:45:00,2128-06-20 11:25:00,2128-06-20 11:25:00,2128-06-10 09:30:00,2128-06-11 08:25:00,None,2,2,True,221,nonfailure,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1375.0,16.0,18.0,0.0,8.1,400.0,543.0,NaN,NaN,8.0,40.0,NaN,CMV/ASSIST/AutoFlow,None,Drager,4.0,72.0,117.0,46.0,66.0,17.0,NaN,None,97.0,NaN


#### gcs추가

In [40]:
#gcs추가

sub1_gcs_notna['charttime'] = pd.to_datetime(sub1_gcs_notna['charttime'])

# Step 1: Group ventset_patient by 'hadm_id' once before the loop
sub1_gcs_grouped = sub1_gcs_notna.groupby('hadm_id')

# Step 2: Initialize an empty result list
optimized_result_list = []

# Step 3: Process each row in subjectlist2_intextevents
for index, row in subjectlist2_intextevents.iterrows():
    hadm_id = row['hadm_id']
    intubationtime = row['intubationtime']
    extubationtime = row['extubationtime']
    
    # Determine start_time based on time difference
    time_diff = extubationtime - intubationtime
    start_time = extubationtime - timedelta(hours=48) if time_diff > timedelta(hours=48) else intubationtime

    # Check if hadm_id exists in ventset_patient_grouped
    if hadm_id in sub1_gcs_grouped.groups:
        # Retrieve relevant group for the hadm_id (faster than filtering the entire dataset)
        group = sub1_gcs_grouped .get_group(hadm_id)
        
        # Step 4: Filter the group by time window only (much smaller subset than the whole dataset)
        filtered_group = group[
            (group['charttime'] >= start_time) &
            (group['charttime'] <= extubationtime)
        ]
        
        # Step 5: Append filtered rows to result
        for _, gcs_row in filtered_group.iterrows():
            new_row = row.copy()
            for col in [
                'charttime', 'gcs',	'gcs_motor','gcs_verbal','gcs_eyes','gcs_unable'
            ]:
                new_row[col] = gcs_row.get(col, None)  # Using get to avoid KeyErrors
            optimized_result_list.append(new_row)

# Step 6: Create the final result DataFrame
gcs_result_df = pd.DataFrame(optimized_result_list)
gcs_result_df

,subject_id,hadm_id,stay_id,los,admittime,intubationtime,extubationtime,extubationcause,dischtime,deathtime,seq_num,total_seq_num,intext_duration,reintubationtime,mvtime,class_code,class,gender,age,charttime,gcs,gcs_motor,gcs_verbal,gcs_eyes,gcs_unable
0,10001884,26184834,37510196,9.171817,2131-01-07 20:39:00,2131-01-11 04:30:00,2131-01-12 17:40:00,Planned Extubation,2131-01-20 05:15:00,2131-01-20 05:15:00,1,4,2230.0,620.0,False,211,failure,F,68,2131-01-11 08:00:00,15.0,1.0,0.0,1.0,1.0
0,10001884,26184834,37510196,9.171817,2131-01-07 20:39:00,2131-01-11 04:30:00,2131-01-12 17:40:00,Planned Extubation,2131-01-20 05:15:00,2131-01-20 05:15:00,1,4,2230.0,620.0,False,211,failure,F,68,2131-01-11 10:00:00,15.0,1.0,0.0,2.0,1.0
0,10001884,26184834,37510196,9.171817,2131-01-07 20:39:00,2131-01-11 04:30:00,2131-01-12 17:40:00,Planned Extubation,2131-01-20 05:15:00,2131-01-20 05:15:00,1,4,2230.0,620.0,False,211,failure,F,68,2131-01-11 12:00:00,15.0,5.0,0.0,3.0,1.0
0,10001884,26184834,37510196,9.171817,2131-01-07 20:39:00,2131-01-11 04:30:00,2131-01-12 17:40:00,Planned Extubation,2131-01-20 05:15:00,2131-01-20 05:15:00,1,4,2230.0,620.0,False,211,failure,F,68,2131-01-11 14:00:00,15.0,5.0,0.0,3.0,1.0
0,10001884,26184834,37510196,9.171817,2131-01-07 20:39:00,2131-01-11 04:30:00,2131-01-12 17:40:00,Planned Extubation,2131-01-20 05:15:00,2131-01-20 05:15:00,1,4,2230.0,620.0,False,211,failure,F,68,2131-01-11 16:00:00,15.0,5.0,0.0,3.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11549,19999840,21033226,38978960,5.297766,2164-09-10 13:47:00,2164-09-13 04:05:00,2164-09-17 12:07:00,NaN,2164-09-17 13:42:00,2164-09-17 13:42:00,2,2,6242.0,NaN,False,22221,death,M,58,2164-09-16 20:21:00,15.0,1.0,0.0,1.0,1.0
11549,19999840,21033226,38978960,5.297766,2164-09-10 13:47:00,2164-09-13 04:05:00,2164-09-17 12:07:00,NaN,2164-09-17 13:42:00,2164-09-17 13:42:00,2,2,6242.0,NaN,False,22221,death,M,58,2164-09-17 00:00:00,15.0,1.0,0.0,1.0,1.0
11549,19999840,21033226,38978960,5.297766,2164-09-10 13:47:00,2164-09-13 04:05:00,2164-09-17 12:07:00,NaN,2164-09-17 13:42:00,2164-09-17 13:42:00,2,2,6242.0,NaN,False,22221,death,M,58,2164-09-17 04:00:00,15.0,1.0,0.0,1.0,1.0
11549,19999840,21033226,38978960,5.297766,2164-09-10 13:47:00,2164-09-13 04:05:00,2164-09-17 12:07:00,NaN,2164-09-17 13:42:00,2164-09-17 13:42:00,2,2,6242.0,NaN,False,22221,death,M,58,2164-09-17 08:08:00,15.0,1.0,0.0,1.0,1.0


In [41]:
# 병합시도
# Concatenate the two dataframes
combined_df = pd.concat([abga_ventset_vital, gcs_result_df], ignore_index=True)

# Group by 'hadm_id' and 'charttime' and combine the columns
abga_ventset_vital_gcs = combined_df.groupby(['hadm_id', 'charttime'], as_index=False).first()

# Add missing columns from both dataframes (in case one is missing a column from the other)
columns_to_add = ['pCO2', 'pO2', 'SaO2', 'HCO3-', 'pH', 'Base excess','respiratory_rate_set', 'respiratory_rate_total',
                'respiratory_rate_spontaneous', 'minute_volume', 'tidal_volume_set',
                'tidal_volume_observed', 'tidal_volume_spontaneous', 'plateau_pressure',
                'peep', 'fio2', 'flow_rate', 'ventilator_mode', 
                'ventilator_mode_hamilton', 'ventilator_type', 'ventilator_scoring',
                'heart_rate','sbp'	,'dbp','mbp','resp_rate',	'temperature',	'temperature_site',	'spo2',	'glucose','gcs','gcs_motor','gcs_verbal','gcs_eyes','gcs_unable']
for col in columns_to_add:
    if col not in abga_ventset_vital_gcs.columns:
        abga_ventset_vital_gcs[col] = float('nan')
abga_ventset_vital_gcs

,hadm_id,charttime,subject_id,los,stay_id,gender,age,dischtime,deathtime,intubationtime,extubationtime,extubationcause,seq_num,total_seq_num,mvtime,class_code,class,reintubationtime,pCO2,pO2,SaO2,HCO3-,pH,Base excess,admittime,intext_duration,respiratory_rate_set,respiratory_rate_total,respiratory_rate_spontaneous,minute_volume,tidal_volume_set,tidal_volume_observed,tidal_volume_spontaneous,plateau_pressure,peep,fio2,flow_rate,ventilator_mode,ventilator_mode_hamilton,ventilator_type,ventilator_scoring,heart_rate,sbp,dbp,mbp,resp_rate,temperature,temperature_site,spo2,glucose,gcs,gcs_motor,gcs_verbal,gcs_eyes,gcs_unable
0,20001361,2143-05-04 17:49:00,14577567,6.046644,33475095,M,30,2143-05-18 16:58:00,NaT,2143-05-04 17:49:00,2143-05-05 08:30:00,Planned Extubation,1,3,True,211,failure,690.0,NaN,NaN,NaN,NaN,NaN,NaN,2143-05-04 14:55:00,881.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,15.0,NaN,0.0,1.0,1.0
1,20001361,2143-05-04 18:00:00,14577567,6.046644,33475095,M,30,2143-05-18 16:58:00,NaT,2143-05-04 17:49:00,2143-05-05 08:30:00,Planned Extubation,1,3,True,211,failure,690.0,NaN,NaN,NaN,NaN,NaN,NaN,2143-05-04 14:55:00,881.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.0,NaN,NaN,None,None,None,NaN,74.0,115.0,80.0,88.0,18.0,NaN,None,89.0,NaN,NaN,NaN,NaN,NaN,NaN
2,20001361,2143-05-04 18:01:00,14577567,6.046644,33475095,M,30,2143-05-18 16:58:00,NaT,2143-05-04 17:49:00,2143-05-05 08:30:00,Planned Extubation,1,3,True,211,failure,690.0,40.0,59.0,NaN,NaN,7.33,-4.0,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,20001361,2143-05-04 18:35:00,14577567,6.046644,33475095,M,30,2143-05-18 16:58:00,NaT,2143-05-04 17:49:00,2143-05-05 08:30:00,Planned Extubation,1,3,True,211,failure,690.0,NaN,NaN,NaN,NaN,NaN,NaN,2143-05-04 14:55:00,881.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,NaN,NaN,NaN,NaN,297.0,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,20001361,2143-05-04 18:46:00,14577567,6.046644,33475095,M,30,2143-05-18 16:58:00,NaT,2143-05-04 17:49:00,2143-05-05 08:30:00,Planned Extubation,1,3,True,211,failure,690.0,NaN,NaN,NaN,NaN,NaN,NaN,2143-05-04 14:55:00,881.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
634081,29999098,2128-06-11 06:00:00,15129243,11.071678,33475199,M,76,2128-06-20 11:25:00,2128-06-20 11:25:00,2128-06-10 09:30:00,2128-06-11 08:25:00,None,1,1,True,11,nonfailure,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2128-06-09 23:45:00,1375.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,NaN,75.0,130.0,50.0,74.0,16.0,NaN,None,97.0,NaN,15.0,5.0,0.0,1.0,1.0
634082,29999098,2128-06-11 07:00:00,15129243,11.071678,33475199,M,76,2128-06-20 11:25:00,2128-06-20 11:25:00,2128-06-10 09:30:00,2128-06-11 08:25:00,None,1,1,True,11,nonfailure,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2128-06-09 23:45:00,1375.0,16.0,18.0,0.0,8.1,400.0,543.0,NaN,NaN,8.0,40.0,NaN,CMV/ASSIST/AutoFlow,None,Drager,4.0,72.0,117.0,46.0,66.0,17.0,NaN,None,97.0,NaN,NaN,NaN,NaN,NaN,NaN
634083,29999098,2128-06-11 07:44:00,15129243,11.071678,33475199,M,76,2128-06-20 11:25:00,2128-06-20 11:25:00,2128-06-10 09:30:00,2128-06-11 08:25:00,None,1,1,True,11,nonfailure,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2128-06-09 23:45:00,1375.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,15.0,5.0,0.0,1.0,1.0
634084,29999098,2128-06-11 08:00:00,15129243,11.071678,33475199,M,76,2128-06-20 11:25:00,2128-06-20 11:25:00,2128-06-10 09:30:00,2128-06-11 08:25:00,None,1,1,True,11,nonfailure,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2128-06-09 23:45:00,1375.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,NaN,72.0,132.0,51.0,73.0,17.0,37.39,Oral,98.0,NaN,

In [42]:
# 컬럼 순서 변경
column_order = [
    'subject_id', 'hadm_id', 'stay_id', 'los', 'gender', 'age','admittime',  'dischtime', 
    'intubationtime', 'extubationtime', 'extubationcause', 'mvtime', 'seq_num', 
    'total_seq_num', 'mvtime_state', 'class_code', 'class', 'charttime',
    
    'heart_rate','sbp','dbp','mbp', 'resp_rate', 'temperature', 'temperature_site', 'spo2', 'glucose', # vital
    'gcs','gcs_motor','gcs_verbal','gcs_eyes','gcs_unable', # gcs
    'respiratory_rate_set', 'respiratory_rate_total', 'respiratory_rate_spontaneous', 'minute_volume', 'tidal_volume_set', # ventset
    'tidal_volume_observed', 'tidal_volume_spontaneous', 'plateau_pressure', 'peep', 'fio2', 'flow_rate', 
    'ventilator_mode', 'ventilator_mode_hamilton', 'ventilator_type', 'ventilator_scoring',
    'pCO2', 'pO2', 'SaO2', 'HCO3-', 'pH', 'Base excess' # abga
    
    
]

# Checking which columns exist in the DataFrame and reordering them
existing_columns = [col for col in column_order if col in abga_ventset_vital_gcs.columns]
abga_ventset_vital_gcs = abga_ventset_vital_gcs[existing_columns]
abga_ventset_vital_gcs

,subject_id,hadm_id,stay_id,los,gender,age,admittime,dischtime,intubationtime,extubationtime,extubationcause,intext_duration,seq_num,total_seq_num,mvtime,class_code,class,charttime,pCO2,pO2,SaO2,HCO3-,pH,Base excess,respiratory_rate_set,respiratory_rate_total,respiratory_rate_spontaneous,minute_volume,tidal_volume_set,tidal_volume_observed,tidal_volume_spontaneous,plateau_pressure,peep,fio2,flow_rate,ventilator_mode,ventilator_mode_hamilton,ventilator_type,ventilator_scoring,heart_rate,sbp,dbp,mbp,resp_rate,temperature,temperature_site,spo2,glucose,gcs,gcs_motor,gcs_verbal,gcs_eyes,gcs_unable
0,14577567,20001361,33475095,6.046644,M,30,2143-05-04 14:55:00,2143-05-18 16:58:00,2143-05-04 17:49:00,2143-05-05 08:30:00,Planned Extubation,881.0,1,3,True,211,failure,2143-05-04 17:49:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,15.0,NaN,0.0,1.0,1.0
1,14577567,20001361,33475095,6.046644,M,30,2143-05-04 14:55:00,2143-05-18 16:58:00,2143-05-04 17:49:00,2143-05-05 08:30:00,Planned Extubation,881.0,1,3,True,211,failure,2143-05-04 18:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.0,NaN,NaN,None,None,None,NaN,74.0,115.0,80.0,88.0,18.0,NaN,None,89.0,NaN,NaN,NaN,NaN,NaN,NaN
2,14577567,20001361,33475095,6.046644,M,30,NaT,2143-05-18 16:58:00,2143-05-04 17:49:00,2143-05-05 08:30:00,Planned Extubation,NaN,1,3,True,211,failure,2143-05-04 18:01:00,40.0,59.0,NaN,NaN,7.33,-4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,14577567,20001361,33475095,6.046644,M,30,2143-05-04 14:55:00,2143-05-18 16:58:00,2143-05-04 17:49:00,2143-05-05 08:30:00,Planned Extubation,881.0,1,3,True,211,failure,2143-05-04 18:35:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,NaN,NaN,NaN,NaN,297.0,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,14577567,20001361,33475095,6.046644,M,30,2143-05-04 14:55:00,2143-05-18 16:58:00,2143-05-04 17:49:00,2143-05-05 08:30:00,Planned Extubation,881.0,1,3,True,211,failure,2143-05-04 18:46:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
634081,15129243,29999098,33475199,11.071678,M,76,2128-06-09 23:45:00,2128-06-20 11:25:00,2128-06-10 09:30:00,2128-06-11 08:25:00,None,1375.0,1,1,True,11,nonfailure,2128-06-11 06:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,NaN,75.0,130.0,50.0,74.0,16.0,NaN,None,97.0,NaN,15.0,5.0,0.0,1.0,1.0
634082,15129243,29999098,33475199,11.071678,M,76,2128-06-09 23:45:00,2128-06-20 11:25:00,2128-06-10 09:30:00,2128-06-11 08:25:00,None,1375.0,1,1,True,11,nonfailure,2128-06-11 07:00:00,NaN,NaN,NaN,NaN,NaN,NaN,16.0,18.0,0.0,8.1,400.0,543.0,NaN,NaN,8.0,40.0,NaN,CMV/ASSIST/AutoFlow,None,Drager,4.0,72.0,117.0,46.0,66.0,17.0,NaN,None,97.0,NaN,NaN,NaN,NaN,NaN,NaN
634083,15129243,29999098,33475199,11.071678,M,76,2128-06-09 23:45:00,2128-06-20 11:25:00,2128-06-10 09:30:00,2128-06-11 08:25:00,None,1375.0,1,1,True,11,nonfailure,2128-06-11 07:44:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,15.0,5.0,0.0,1.0,1.0
634084,15129243,29999098,33475199,11.071678,M,76,2128-06-09 23:45:00,2128-06-20 11:25:00,2128-06-10 09:30:00,2128-06-11 08:25:00,None,1375.0,1,1,True,11,nonfailure,2128-06-11 08:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,NaN,72.0,132.0,51.0,73.0,17.0,37.39,Oral,98.0,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
pd.set_option('display.max_info_rows', None)

In [44]:
abga_ventset_vital_gcs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 634086 entries, 0 to 634085
Data columns (total 53 columns):
 #   Column                        Non-Null Count   Dtype         
---  ------                        --------------   -----         
 0   subject_id                    634086 non-null  int64         
 1   hadm_id                       634086 non-null  int64         
 2   stay_id                       634086 non-null  int64         
 3   los                           617387 non-null  float64       
 4   gender                        634086 non-null  object        
 5   age                           634086 non-null  int64         
 6   admittime                     603320 non-null  datetime64[ns]
 7   dischtime                     634086 non-null  datetime64[ns]
 8   intubationtime                634086 non-null  datetime64[ns]
 9   extubationtime                634086 non-null  datetime64[ns]
 10  extubationcause               426441 non-null  object        
 11  intext_durati

In [ ]:
# 최종 테이블 저장
abga_ventset_vital_gcs.to_csv('./data/abga_ventset_vital_gcs.csv') 